##Importing the libraries

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

##Loading the Dataset

In [2]:
# Load the dataset
df = pd.read_csv('train.csv')

##a)  Look for the missing values in all the columns and either impute them (replace with mean, median, or mode) or drop them. Justify your action for this task.

In [3]:
missing_summary = df.isnull().sum()
print("Missing Values:\n", missing_summary)

Missing Values:
 Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64


In [4]:
threshold = 0.5

# Calculate the threshold in terms of number of rows
row_count = len(df)
column_threshold = row_count * threshold

# Drop columns where missing values > 50%
df_cleaned = df.loc[:, df.isnull().sum() <= column_threshold]

# Show dropped columns
dropped_columns = df.columns[df.isnull().sum() > column_threshold]
print(f"Columns dropped (more than 50% missing): {list(dropped_columns)}")

Columns dropped (more than 50% missing): ['New_Price']


In [5]:
# Display remaining missing values
print( df_cleaned.isnull().sum())

Unnamed: 0            0
Name                  0
Location              0
Year                  0
Kilometers_Driven     0
Fuel_Type             0
Transmission          0
Owner_Type            0
Mileage               2
Engine               36
Power                36
Seats                38
Price                 0
dtype: int64


##b) Remove the units from some of the attributes and only keep the numerical values (for example remove kmpl from “Mileage”, CC from “Engine”, bhp from “Power”, and lakh from “New_price”).

In [6]:
# Function to extract numeric part using regex
def extract_numeric(value):
    if pd.isnull(value):
        return np.nan
    match = re.search(r"[\d.]+", str(value))
    return float(match.group()) if match else np.nan

# Apply to relevant columns
df['Mileage'] = df['Mileage'].apply(extract_numeric)
df['Engine'] = df['Engine'].apply(extract_numeric)
df['Power'] = df['Power'].apply(extract_numeric)

# Special case for 'New_Price' (handle 'null' and remove 'Lakh')
df['New_Price'] = df['New_Price'].replace('null', np.nan)
df['New_Price'] = df['New_Price'].apply(extract_numeric)


In [7]:
# Preview cleaned data
print(df[['Mileage', 'Engine', 'Power', 'New_Price']].head())

   Mileage  Engine   Power  New_Price
0    19.67  1582.0  126.20        NaN
1    13.00  1199.0   88.70       8.61
2    20.77  1248.0   88.76        NaN
3    15.20  1968.0  140.80        NaN
4    23.08  1461.0   63.10        NaN


##C) Change the categorical variables (“Fuel_Type” and “Transmission”) into numerical one hot encoded value.  

In [8]:
# One-hot encode 'Fuel_Type' and 'Transmission'
df_encoded = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'], drop_first=True)

# Show the new columns
print(df_encoded.columns)
print(df_encoded.head())


Index(['Unnamed: 0', 'Name', 'Location', 'Year', 'Kilometers_Driven',
       'Owner_Type', 'Mileage', 'Engine', 'Power', 'Seats', 'New_Price',
       'Price', 'Fuel_Type_Electric', 'Fuel_Type_Petrol',
       'Transmission_Manual'],
      dtype='object')
   Unnamed: 0                              Name    Location  Year  \
0           1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1           2                      Honda Jazz V     Chennai  2011   
2           3                 Maruti Ertiga VDI     Chennai  2012   
3           4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4           6            Nissan Micra Diesel XV      Jaipur  2013   

   Kilometers_Driven Owner_Type  Mileage  Engine   Power  Seats  New_Price  \
0              41000      First    19.67  1582.0  126.20    5.0        NaN   
1              46000      First    13.00  1199.0   88.70    5.0       8.61   
2              87000      First    20.77  1248.0   88.76    7.0        NaN   
3              4067

##d) Create one more feature and add this column to the dataset (you can use mutate function in R for this). For example, you can calculate the current age of the car by subtracting “Year” value from the current year.   

In [9]:
# Get the current year
current_year = datetime.now().year

# Create new feature: Car_Age
df['Car_Age'] = current_year - df['Year']

# Show the updated dataframe
print(df[['Year', 'Car_Age']].head())

   Year  Car_Age
0  2015       10
1  2011       14
2  2012       13
3  2013       12
4  2013       12


##e) Perform select, filter, rename, mutate, arrange and summarize with group by operations (or their equivalent operations in python) on this dataset.

###Select

In [10]:
df_selected = df[['Name', 'Location', 'Fuel_Type', 'Transmission', 'Car_Age', 'Mileage', 'Price']]
print("\nSelected columns:\n", df_selected.head())



Selected columns:
                                Name    Location Fuel_Type Transmission  \
0  Hyundai Creta 1.6 CRDi SX Option        Pune    Diesel       Manual   
1                      Honda Jazz V     Chennai    Petrol       Manual   
2                 Maruti Ertiga VDI     Chennai    Diesel       Manual   
3   Audi A4 New 2.0 TDI Multitronic  Coimbatore    Diesel    Automatic   
4            Nissan Micra Diesel XV      Jaipur    Diesel       Manual   

   Car_Age  Mileage  Price  
0       10    19.67  12.50  
1       14    13.00   4.50  
2       13    20.77   6.00  
3       12    15.20  17.74  
4       12    23.08   3.50  


###Filter

In [11]:
df_filtered = df[df['Car_Age'] < 5]
print("\nFiltered cars (less than 5 years old):\n", df_filtered[['Name', 'Car_Age']].head())


Filtered cars (less than 5 years old):
 Empty DataFrame
Columns: [Name, Car_Age]
Index: []


###Rename

In [12]:
df_renamed = df.rename(columns={'Price': 'Selling_Price'})
print("\nRenamed column 'Price' to 'Selling_Price':\n", df_renamed[['Selling_Price']].head())



Renamed column 'Price' to 'Selling_Price':
    Selling_Price
0          12.50
1           4.50
2           6.00
3          17.74
4           3.50


###Mutate

In [13]:
df_renamed['Mileage_per_Year'] = df_renamed['Kilometers_Driven'] / df_renamed['Car_Age']
print("\nNew column 'Mileage_per_Year':\n", df_renamed[['Kilometers_Driven', 'Car_Age', 'Mileage_per_Year']].head())



New column 'Mileage_per_Year':
    Kilometers_Driven  Car_Age  Mileage_per_Year
0              41000       10       4100.000000
1              46000       14       3285.714286
2              87000       13       6692.307692
3              40670       12       3389.166667
4              86999       12       7249.916667


###Arrange

In [14]:
df_arranged = df_renamed.sort_values(by='Selling_Price', ascending=False)
print("\nArranged by Selling_Price (Descending):\n", df_arranged[['Name', 'Selling_Price']].head())


Arranged by Selling_Price (Descending):
                                              Name  Selling_Price
3952  Land Rover Range Rover 3.0 Diesel LWB Vogue         160.00
5620                   Lamborghini Gallardo Coupe         120.00
5752                       Jaguar F Type 5.0 V8 S         100.00
1457              Land Rover Range Rover Sport SE          97.07
1917                           BMW 7 Series 740Li          93.67


###Summarize with Group By

In [15]:
df_summary = df_renamed.groupby('Location').agg(
    Total_Cars=('Name', 'count'),
    Avg_Selling_Price=('Selling_Price', 'mean'),
    Max_Selling_Price=('Selling_Price', 'max')
).reset_index()

print("\nGrouped Summary by Location:\n", df_summary)


Grouped Summary by Location:
       Location  Total_Cars  Avg_Selling_Price  Max_Selling_Price
0    Ahmedabad         218           8.567248              85.00
1    Bangalore         352          13.482670              93.00
2      Chennai         476           7.958340              70.00
3   Coimbatore         631          15.160206              93.67
4        Delhi         540           9.881944             120.00
5    Hyderabad         710           9.997423             160.00
6       Jaipur         403           5.916725              42.50
7        Kochi         640          11.309109              97.07
8      Kolkata         525           5.733924              75.00
9       Mumbai         762           9.592546              75.00
10        Pune         590           6.951000              68.00
